In [1]:
_ = '''
transaction = contract.newEvent({
    'currencyPair': 'XTZ-USD',
    'targetDynamics': 1000000,
    'betsCloseTime': int(time.time()) + 3600,
    'measurePeriod': 3600,
    'liquidityPercent': 10000,
}).as_transaction()

transaction.json_payload()#.hash()
'''

### Juster-Maker:
Tool allowing to run events in cycle

In [2]:
from pytezos import pytezos
import time
from pprint import pprint
from utility import repeat_until_succeed
from bulk_sender import BulkSender
from event_emitter import EventCreationEmitter


# pytezos = pytezos.using(shell='https://florencenet-tezos.giganode.io')
pytezos = pytezos.using(shell='https://api.tez.ie/rpc/florencenet')

In [3]:
_ = '''
CONFIG = {
    'events': [],
    'juster_address': 'KT1CepDBrMg73d7LHm773KAsunjAjgLYituP'
}
'''

In [4]:
address = 'KT1CepDBrMg73d7LHm773KAsunjAjgLYituP'
client = pytezos.using(
    key='../2021-05-27-tests-modelling/test-keys/tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ.json'
)
contract = client.contract(address)

In [5]:
import time
time.time()

1626719355.6442986

In [6]:
from utility import date_to_timestamp

### TODO:
- starts_from parameter for events creators so it would be possible to bind events to the time slots

In [7]:
from asyncio import Queue

# TODO: what would happen if queue overflow?
MAX_QUEUE_SIZE = 50

operations_queue = Queue(MAX_QUEUE_SIZE)

events_params = [
    {
        'currency_pair': currency_pair,
        'target_dynamics': 1_000_000,
        'bets_period': period,
        'measure_period': period,
        'liquidity_percent': 10_000,
        'expiration_fee': 100_000,
        'measure_start_fee': 100_000
    } for currency_pair in ['XTZ-USD', 'BTC-USD']
    for period in [600, 900]
]

In [8]:
# for each event_params EventCreationEmitter is created:
emitters = [
    EventCreationEmitter(
        period=20,
        contract=contract,
        operations_queue=operations_queue,
        event_params=params,
        next_at=int(date_to_timestamp('2021-07-19 21:30:00')))
    for params in events_params
]

emitters.append(
    BulkSender(period=60, client=client, operations_queue=operations_queue)
)

In [9]:
for emitter in emitters:
    emitter.run()

time has not come, waiting a little: 42.10192894935608 secs
time has not come, waiting a little: 42.101845264434814 secs
time has not come, waiting a little: 42.10180401802063 secs
time has not come, waiting a little: 42.10175895690918 secs
time has not come, waiting a little: 22.080868005752563 secs
time has not come, waiting a little: 22.08076572418213 secs
time has not come, waiting a little: 22.080731868743896 secs
time has not come, waiting a little: 22.08070468902588 secs
time has not come, waiting a little: 2.06337308883667 secs
time has not come, waiting a little: 2.06321382522583 secs
time has not come, waiting a little: 2.0631463527679443 secs
time has not come, waiting a little: 2.0630862712860107 secs
created newEvent transaction with parameters: {'currencyPair': 'XTZ-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626720000, 'measurePeriod': 600, 'liquidityPercent': 10000}
next event at: 1626720000
created newEvent transaction with parameters: {'currencyPair': 'XTZ-USD'

### TODO: OK, в общем проще всего мне кажется вот так:
    - для каждой линейки ивентов создаётся свой event_emitter, он проверяет что пришло время запустить ивет сравнивая текущее значение таймера с таргетом. И после запуска ивента он переставляет таймер дальше.
    - с определённой частотой (например раз в 20 секунд) таймер проверяется и если время больше чем в таймере - транза с новым ивентом закидывается в очередь и подхватывается BulkSender

### TODO: maybe make next_event func/iterator that returns time and params for the next events

### TODO: need to have some tests for LoopExecutor:
- for example that it runned function K times in given time

### TODO: need to test all this code before running in florencenet

In [10]:
for emitter in emitters:
    emitter.stop()